In [ ]:
# Test Pareto Optimizer

import pandas as pd
import json
from typing import Dict, Any, List
import numpy as np
from datetime import datetime, timedelta
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs, Trial
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer, ParetoData
from robyn.data.entities.enums import DependentVarType, PaidMediaSigns, OrganicSigns, ContextSigns

from utils.data_mapper import import_data, load_data_from_json


In [ ]:
# Load data from JSON
loaded_data = load_data_from_json(
    "/Users/funny/Documents/git/Robyn/python/src/tutorials/resources/test_data.json"
    # "/Users/funny/Downloads/exported_data.json"
)
imported_data = import_data(loaded_data)
model_outputs = imported_data["model_outputs"]
display((model_outputs.hyper_bound_ng))


In [3]:

mmm_data = imported_data["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

model_outputs = imported_data["model_outputs"]
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = imported_data["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = imported_data["featurized_mmm_data"]

holidays_data = imported_data["holidays_data"]

In [4]:

# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, model_outputs, hyperparameters, featurized_mmm_data, holidays_data)

In [5]:

# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=3)


In [7]:
# def serialize_data(aggregated_data: dict, pareto_data: ParetoData) -> str:
#     # Convert aggregated_data DataFrames to dictionaries
#     json_aggregated_data = {
#         key: value.to_dict(orient="records") if value is not None else None for key, value in aggregated_data.items()
#     }

#     # Convert pareto_data to dictionary
#     json_pareto_data = {
#             "decomp_spend_dist": pareto_data.decomp_spend_dist.to_dict(orient="records"),
#             "result_hyp_param": pareto_data.result_hyp_param.to_dict(orient="records"),
#             "x_decomp_agg": pareto_data.x_decomp_agg.to_dict(orient="records"),
#             "pareto_fronts": pareto_data.pareto_fronts,
#         }

#     # Combine both into a single dictionary
#     combined_data = {
#         "aggregated_data": json_aggregated_data,
#         "pareto_data": json_pareto_data,
#     }

#     # Serialize to JSON
#     return json.dumps(combined_data)

# def export_to_json_file(json_string: str, filename: str) -> None:
#     """Export the JSON string to a file."""
#     with open(filename, 'w') as json_file:
#         json_file.write(json_string)

# # Example usage
# # Assuming pareto_result is defined and contains the necessary data
# aggregated_data = pareto_result["aggregated_data"]
# pareto_data = pareto_result["pareto_data"]

# # Serialize
# json_output = serialize_data(aggregated_data, pareto_data)
# print("Serialized JSON:", json_output)

# # Export to JSON file
# export_to_json_file(json_output, 'output_data.json')


In [8]:
def import_from_json_file(filename: str) -> str:
    """Read JSON data from a file."""
    with open(filename, 'r') as json_file:
        return json_file.read()


def deserialize_data(json_string: str) -> Dict[str, Any]:
    # Parse the JSON string back into a dictionary
    data = json.loads(json_string)

    # Reconstruct aggregated_data
    aggregated_data = {
        key: pd.DataFrame(value) for key, value in data["aggregated_data"].items()
    }

    # Reconstruct pareto_data
    pareto_data_dict = data["pareto_data"]
    pareto_data = ParetoData(
        decomp_spend_dist=pd.DataFrame(pareto_data_dict["decomp_spend_dist"]),
        result_hyp_param=pd.DataFrame(pareto_data_dict["result_hyp_param"]),
        x_decomp_agg=pd.DataFrame(pareto_data_dict["x_decomp_agg"]),
        pareto_fronts=pareto_data_dict["pareto_fronts"],
    )

    return {
        "aggregated_data": aggregated_data,
        "pareto_data": pareto_data,
    }

# Import from JSON file
imported_json_output = import_from_json_file('output_data.json')

# Deserialize
deserialized_data = deserialize_data(imported_json_output)

In [ ]:
pareto_results = pareto_optimizer._generate_plot_data(deserialized_data["aggregated_data"], deserialized_data["pareto_data"])

In [ ]:
# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto fronts: {pareto_result.pareto_fronts}")
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param.head())

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg.head())

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(pareto_result.pareto_fronts, int), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")